In [13]:
import json
import os.path as osp
import matplotlib.pyplot as plt
from collections import OrderedDict

base_path = "../VIST/sis"
train_data = json.load(open(osp.join(base_path, "train.story-in-sequence.json")))
val_data = json.load(open(osp.join(base_path, "val.story-in-sequence.json")))
test_data = json.load(open(osp.join(base_path, "test.story-in-sequence.json")))

prefix = ["train", "val", "test"]
whole_album2im = {}
for i, data in enumerate([train_data, val_data, test_data]):
    album2im = {}
    for im in data['images']:
        if im['album_id'] not in album2im:
            album2im[im['album_id']] = [im['id']]
        else:
            if im['id'] not in album2im[im['album_id']]:
                album2im[im['album_id']].append(im['id'])
    whole_album2im[prefix[i]] = album2im

whole_album = {}
story_lines = {}
whole_lines = {}
story_line_count = 0
whole_line_count = 0
for i, data in enumerate([train_data, val_data, test_data]):
    album_mapping = {}
    for annot_new in data["annotations"]:
        annot = annot_new[0]
        assert len(annot_new) == 1
        text = annot['text'].encode('utf8')
        if annot['story_id'] not in album_mapping:
            album_mapping[annot['story_id']] = {"text_index": [story_line_count], "flickr_id": [annot['photo_flickr_id']], "length": 1, 
                                                "album_id": annot['album_id'], "album_flickr_id": whole_album2im[prefix[i]][annot['album_id']],
                                                "whole_text_index": whole_line_count, "origin_text": text}
            story_lines[annot['story_id']] = [{"index": story_line_count, "text": text.split()}]
            whole_lines[annot['story_id']] = {"index": whole_line_count, "text": text.split()}
            whole_line_count +=1
        else:
            album_mapping[annot['story_id']]["text_index"].append(story_line_count)
            album_mapping[annot['story_id']]["flickr_id"].append(annot['photo_flickr_id'])
            album_mapping[annot['story_id']]["length"] += 1
            story_lines[annot['story_id']].append({"index": story_line_count, "text": text.split()})
            whole_lines[annot['story_id']]["text"].extend(text.split())
            album_mapping[annot['story_id']]["origin_text"] += " " + text 
        story_line_count += 1
    whole_album[prefix[i]] = album_mapping

new_story_lines = [] 
for l in story_lines.values():
    for li in l:
        new_story_lines.append(li)
story_lines = new_story_lines
whole_lines = whole_lines.values()

story_lines = [r['text'] for r in sorted(story_lines, key=lambda thing: thing['index'])]
whole_lines = [r['text'] for r in sorted(whole_lines, key=lambda thing: thing['index'])]

print len(story_lines)
print len(whole_lines)

from collections import Counter
import numpy
cnt = Counter()
for l in story_lines:
    words = l
    for w in words:
        cnt[w] += 1
words2id = {}
idx = 2
for k, v in cnt.most_common():
    if v > 5:
        words2id[k] = idx
        idx += 1
words2id["<EOS>"] = 0
words2id["<UNK>"] = 1
id2words = {v:k for k,v in words2id.iteritems()}
print len(id2words)

whole_album["words2id"] = words2id
whole_album["id2words"] = {v:k for k,v in words2id.iteritems()}

id_story_lines = []
for l in story_lines:
    s = [words2id[w] if w in words2id else 1 for w in l]
    id_story_lines.append(s)

id_whole_lines = []
for l in whole_lines:
    s = [words2id[w] if w in words2id else 1 for w in l]
    id_whole_lines.append(s)

new_id_whole_lines = []
specify_longest = 105
for i in range(len(id_whole_lines)):
    cur_len = len(id_whole_lines[i])
    if cur_len < specify_longest:
        new_id_whole_lines.append(id_whole_lines[i] + [0] * (specify_longest - cur_len))
    else:
        new_id_whole_lines.append(id_whole_lines[i][:specify_longest-1] + [0])

data = numpy.asarray(new_id_whole_lines)
import h5py
f = h5py.File("full_story.h5", "w")
f.create_dataset("story", data=data)
f.close()

new_id_story_lines = []
specify_longest = 30
for i in range(len(id_story_lines)):
    cur_len = len(id_story_lines[i])
    if cur_len < specify_longest:
        new_id_story_lines.append(id_story_lines[i] + [0] * (specify_longest - cur_len))
    else:
        new_id_story_lines.append(id_story_lines[i][:specify_longest-1] + [0])

data = numpy.asarray(new_id_story_lines, "int32")
import h5py
f = h5py.File("story.h5", "w")
f.create_dataset("story", data=data)
f.close()

251000
50200
9837


In [15]:
for p in prefix:
    path = "/mnt/sshd/wenhuchen/VIST/images_256/{}/".format(p)
    deletables = []
    for story_id, story in whole_album[p].iteritems():
        d = [osp.exists(osp.join(path, "{}.jpg".format(_))) for _ in story["flickr_id"]]
        if sum(d) < 5:
            print "deleting {}".format(story_id)
            deletables.append(story_id)
        else:
            pass
    for i in deletables:
        del whole_album[p][i]

deleting 38602
deleting 38603
deleting 8084
deleting 8085
deleting 8080
deleting 8081
deleting 8082
deleting 8083
deleting 8089
deleting 7462
deleting 7460
deleting 7464
deleting 36709
deleting 36708
deleting 36707
deleting 36706
deleting 36705
deleting 18994
deleting 18992
deleting 18993
deleting 18990
deleting 18991
deleting 36659
deleting 36658
deleting 36655
deleting 36657
deleting 36656
deleting 8053
deleting 8052
deleting 8051
deleting 8050
deleting 8054
deleting 36756
deleting 36757
deleting 36758
deleting 8020
deleting 8024
deleting 24331
deleting 24330
deleting 24333
deleting 24332
deleting 24334
deleting 18610
deleting 18613
deleting 25587
deleting 25586
deleting 25588
deleting 33058
deleting 33059
deleting 33055
deleting 33056
deleting 33057
deleting 7388
deleting 7389
deleting 7387
deleting 7385
deleting 41915
deleting 41919
deleting 49042
deleting 49044
deleting 49043
deleting 49040
deleting 49041


In [16]:
flickr_story_map = {}
for pre in prefix:
    album = whole_album[pre]
    for k, v in album.iteritems():
        indexes = v['text_index']
        for i, flickr_id in enumerate(v['flickr_id']):
            if flickr_id not in flickr_story_map:
                flickr_story_map[flickr_id] = [indexes[i]]
            else:
                flickr_story_map[flickr_id].append(indexes[i])

In [1]:
length_distribution = [len(s) for s in whole_lines]
result = plt.hist(length_distribution, bins='auto', cumulative=True, normed=1)
#plt.show()
length_distribution = [len(s) for s in story_lines]
result = plt.hist(length_distribution, bins='auto', cumulative=True, normed=1)
#plt.hist(length_distribution, bins='auto')
#plt.show()

NameError: name 'whole_lines' is not defined

In [18]:
import json
import os.path as osp

base_path = "../VIST/dii"
train_data = json.load(open(osp.join(base_path, "train.description-in-isolation.json")))
val_data = json.load(open(osp.join(base_path, "val.description-in-isolation.json")))
test_data = json.load(open(osp.join(base_path, "test.description-in-isolation.json")))

mapping = {}
mapping_original = {}
text_list = []
text_list_count = 0
unknown_words = 0
total_words = 0
with_story = 0
no_story = 0
for i, data in enumerate([train_data, val_data, test_data]):
    mapping[prefix[i]] = {}
    mapping_original[prefix[i]] = {}
    for l in data['annotations']:
        if l[0]['photo_flickr_id'] not in mapping[prefix[i]]:
            if l[0]['photo_flickr_id'] in flickr_story_map:
                stories =  flickr_story_map[l[0]['photo_flickr_id']]
            else:
                stories = [-1]
            mapping[prefix[i]][l[0]['photo_flickr_id']] = {'caption': [text_list_count], 'story': stories}
            mapping_original[prefix[i]][l[0]['photo_flickr_id']] = [l[0]['text']]
        else:
            mapping[prefix[i]][l[0]['photo_flickr_id']]['caption'].append(text_list_count)
            mapping_original[prefix[i]][l[0]['photo_flickr_id']].append(l[0]['text'])
        text_list_count += 1
        assert len(l) == 1
        s = []
        for w in l[0]['text'].split(" "):
            if w in words2id:
                s.append(words2id[w])  
            else:
                s.append(1)
                unknown_words += 1
            total_words += 1
        text_list.append(s)
print "unknown words percent is {}".format(unknown_words / (total_words + 0.0))
new_text_list = []
specify_longest = 20
for i in range(len(text_list)):
    cur_len = len(text_list[i])
    if cur_len < specify_longest:
        new_text_list.append(text_list[i] + [0] * (specify_longest - cur_len))
    else:
        new_text_list.append(text_list[i][:specify_longest - 1] + [0]) 

for p in prefix:
    path = "/mnt/sshd/wenhuchen/VIST/images_256/{}/".format(p)
    deletables = []
    for flickr_id, story in mapping[p].iteritems():
        if not osp.exists(osp.join(path, "{}.jpg".format(flickr_id))):
            deletables.append(flickr_id)
    for i in deletables:
        del mapping[p][i]
        del mapping_original[p][i]
        
whole_album["image2caption"] = mapping
whole_album["image2caption_original"] = mapping_original

with open("story_line.json", 'w') as f:
    json.dump(whole_album, f)

text_array = numpy.asarray(new_text_list, dtype='int32')
import h5py
f = h5py.File("description.h5", 'w')
f.create_dataset("story", data=text_array)
f.close()

unknown words percent is 0.0243577608952


In [182]:
import json
import os.path as osp
base_path = "../VIST/dii"

val_data = json.load(open(osp.join(base_path, "val.description-in-isolation.json")))
with open("val_desc_reference", "w") as f:
    for l in val_data['annotations']:
        print >> f, "{}\t{}".format(l[0]['photo_flickr_id'], l[0]['text'])

import h5py
f = h5py.File("full_story.h5", "r")
print f['story'][0]

f = h5py.File("story.h5", "r")
print f['story'].shape

f = open("story_line.json", 'r')
data = json.load(f)
print len(data['id2words'])

[  36 2293  285   12  141    7   97    5   52 2811    6 1807   14    4
   95 1441    2   31   10   90    5   96    3   35    5   79   32    6
  264  340  649    2    5   55    3  633 2749    8    3 3920    9    1
   32  115    3 8139    8    3  940    2    6  340 1268 3617    9 1716
  311  291 1381    2   47  121   81    8  340  649   31   10   85 1278
   13 1032    2    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0]
(251000, 30)
9837


In [191]:
import os
import numpy

zero_fc = numpy.zeros((2048, ), "float32")
zero_conv = numpy.zeros((2048, 7, 7), "float32")

train_fc_base = "/mnt/sshd/xwang/VIST/feature/train/fc"
train_conv_base = "/mnt/sshd/xwang/VIST/feature/train/conv"
train_name1 = [l.split(".")[0] for l in os.listdir(train_fc_base)]

train_image_base = "/mnt/sshd/wenhuchen/VIST/images/train"
train_name2 = [l.split(".")[0] for l in os.listdir(train_image_base)]

rest = set(train_name2) - set(train_name1)
for image in rest:
    numpy.save(os.path.join(train_fc_base, "{}.npy".format(image)), zero_fc) 
    numpy.save(os.path.join(train_conv_base, "{}.npy".format(image)), zero_conv) 

val_fc_base = "/mnt/sshd/xwang/VIST/feature/val/fc"
val_conv_base = "/mnt/sshd/xwang/VIST/feature/val/conv"
val_name1 = [l.split(".")[0] for l in os.listdir(val_fc_base)]

val_image_base = "/mnt/sshd/wenhuchen/VIST/images/val"
val_name2 = [l.split(".")[0] for l in os.listdir(val_image_base)]

rest = set(val_name2) - set(val_name1)
for image in rest:
    numpy.save(os.path.join(val_fc_base, "{}.npy".format(image)), zero_fc) 
    numpy.save(os.path.join(val_conv_base, "{}.npy".format(image)), zero_conv) 

test_fc_base = "/mnt/sshd/xwang/VIST/feature/test/fc"
test_conv_base = "/mnt/sshd/xwang/VIST/feature/test/conv"
test_name1 = [l.split(".")[0] for l in os.listdir(test_fc_base)]

test_image_base = "/mnt/sshd/wenhuchen/VIST/images/test"
test_name2 = [l.split(".")[0] for l in os.listdir(test_image_base)]

rest = set(test_name2) - set(test_name1)
for image in rest:
    numpy.save(os.path.join(test_fc_base, "{}.npy".format(image)), zero_fc) 
    numpy.save(os.path.join(test_conv_base, "{}.npy".format(image)), zero_conv) 

In [3]:
import json

with open("story_line.json", 'r') as f: 
    data = json.load(f)

print len(data['image2caption']['train'])
print len(data['train'])

40141
40098
